In [1]:
from numpy import *
from matplotlib import *
from pylab import *
from scipy import *
from numpy import*
import csv
import xlrd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

rc('text',usetex = True)
rc('font', family='serif')

In [2]:
import csv

H_Fe = []
B_Fe = []

csv.register_dialect('myDialect', delimiter = ';')
with open('BH_S235JR.csv', 'r') as csvFile:
    reader = csv.reader(csvFile, dialect='myDialect')
    for row in reader:
        H_Fe.append((row[0]))
        B_Fe.append((row[1]))
csvFile.close()

In [3]:
alpha_mag = (45*pi/180)/3
Rg = 190
Ro = 155
Ri = 30
Bsat = 1.8
N = 3
mu_rec = 1.1
mu_0 = 4*pi*10**-7
Br = 1.41
alpha_ar = (10*pi/180)
alpha_dente = (10*pi/180)
aux = np.arange(0,N)
K_disp = 0.1
L = 160

k_disp1 = 1
k_disp2 = 1
k_disp3 = 1
k_a = 1

In [148]:
aux = np.arange(0,N)
B_target = 1
Bgap = np.zeros(N)

Bgap[0] = B_target/10
Bgap[1] = B_target
Bgap[2] = B_target

In [149]:
B_r = Bgap*(3*alpha_mag/2)*(Rg+Ro)/(Ro - Ri)

In [150]:
for i in range(1,len(H_Fe)-1):
    if B_r[0] <np.float(B_Fe[i+1]) and B_r[0]>np.float(B_Fe[i]):
        Hr1 = np.float(H_Fe[i]) + (np.float(H_Fe[i+1])-np.float(H_Fe[i]))*(B_r[0]-np.float(B_Fe[i]))/(np.float(B_Fe[i+1])-np.float(B_Fe[i]))
    if B_r[1] <np.float(B_Fe[i+1]) and B_r[1]>np.float(B_Fe[i]):
        Hr2 = np.float(H_Fe[i]) + (np.float(H_Fe[i+1])-np.float(H_Fe[i]))*(B_r[1]-np.float(B_Fe[i]))/(np.float(B_Fe[i+1])-np.float(B_Fe[i]))
    if B_r[2] <np.float(B_Fe[i+1]) and B_r[2]>np.float(B_Fe[i]):
        Hr3 = np.float(H_Fe[i]) + (np.float(H_Fe[i+1])-np.float(H_Fe[i]))*(B_r[2]-np.float(B_Fe[i]))/(np.float(B_Fe[i+1])-np.float(B_Fe[i]))
    if Bsat <np.float(B_Fe[i+1]) and Bsat>np.float(B_Fe[i]):
        Hsat = np.float(H_Fe[i]) + (np.float(H_Fe[i+1])-np.float(H_Fe[i]))*(Bsat-np.float(B_Fe[i]))/(np.float(B_Fe[i+1])-np.float(B_Fe[i]))

In [151]:
H_g = Bgap/mu_0

Hg1 = H_g[0]
Hg2 = H_g[1]
Hg3 = H_g[2]

Hc1 = Hsat
Hc2 = Hsat
Hc3 = Hsat

Hd = Hsat
Hp = Hsat

def equations(X):
    Hm1,Hm2,Hm3,Rs1,Rs2,Rs3,Rc1,Rc2,Rc3,Ha,Rso = X
    #Áreas:
    
    Sr1 = (Ro-Ri)/3*L
    Sr2 = Sr1
    Sr3 = Sr1
    
    Sg1 = alpha_mag*(Rg+Ro)/2*L
    Sg2 = Sg1
    Sg3 = Sg1
    
    Sm1 = alpha_mag*(Rs1+Rg)/2*L
    Sm2 = alpha_mag*(Rs2+Rg)/2*L
    Sm3 = alpha_mag*(Rs3+Rg)/2*L
    
    Sc1 = (Rc1-Rs1)/4*L
    Sc2 = (Rc2-Rs2)/4*L
    Sc3 = (Rc3-Rs3)/4*L
    
    Sa = (k_a*Rso-Rg)*L
    Sb = Sa
    
    alpha_0 = pi/2 - alpha_ar- 3*alpha_mag - alpha_dente
    
    #Comprimentos:
    Lr1 = 2*(alpha_0 + alpha_dente+alpha_ar+1*alpha_mag/2)*(Ri+3*(Ro-Ri)/4) + 2*1*(Ro-Ri)/4
    Lr2 = 2*(alpha_0 + alpha_dente+alpha_ar+3*alpha_mag/2)*(Ri+2*(Ro-Ri)/4) + 2*2*(Ro-Ri)/4
    Lr3 = 2*(alpha_0 + alpha_dente+alpha_ar+5*alpha_mag/2)*(Ri+1*(Ro-Ri)/4) + 2*3*(Ro-Ri)/4
    
    Lg1 = Rg - Ro
    Lg2 = Lg1
    Lg3 = Lg1
    
    Lm1 = Rs1 - Rg
    Lm2 = Rs2 - Rg
    Lm3 = Rs3 - Rg
        
    Lc1 = 2*(alpha_0 + alpha_dente+alpha_ar+1*alpha_mag/2)*(Rs1+1*(Rc1-Rs1)/4) + 2*1*(Rc1-Rs1)/4
    Lc2 = 2*(alpha_0 + alpha_dente+alpha_ar+3*alpha_mag/2)*(Rs2+2*(Rc2-Rs2)/4) + 2*2*(Rc2-Rs2)/4
    Lc3 = 2*(alpha_0 + alpha_dente+alpha_ar+5*alpha_mag/2)*(Rs3+3*(Rc3-Rs3)/4) + 2*3*(Rc3-Rs3)/4
    
    Lp = 2*(alpha_0+alpha_dente)*(Rso+Rg)/2
    La = alpha_ar*(Rso+Rg)/2 
    Lb=La    
    
    return(
        2*Hm1*Lm1 + 2*Lg1*Hg1+ Hr1*Lr1+ Lc1*Hc1,
        2*Hm2*Lm2 + 2*Lg2*Hg2+ Hr2*Lr2+ Lc2*Hc2,
        2*Hm3*Lm3 + 2*Lg3*Hg3+ Hr3*Lr3+ Lc3*Hc3,
        
        2*Lm1*Hm1 + 2*Ha*La + Lp*Hp + Hc1*Lc1,
        
        k_disp1*(mu_rec*mu_0*Hm1+Br)*Sm1 - Bgap[0]*Sg1 - mu_0*Ha*Sa,
        k_disp2*(mu_rec*mu_0*Hm2+Br)*Sm2 - Bgap[1]*Sg2,
        k_disp3*(mu_rec*mu_0*Hm3+Br)*Sm3 - Bgap[2]*Sg3,
        
        #B_r1*Sr1 - Bgap[0]*Sg1,
        #B_r2*Sr2 - Bgap[1]*Sg2,
        #B_r3*Sr3 - Bgap[2]*Sg3,
        
        #mu_0*Ha*(k_a*Rso-Rg)*L - B_sat*(Rso-Rg)*L,
        mu_0*Ha- Bsat,
        
        k_disp1*(mu_rec*mu_0*Hm1+Br)*Sm1 - Bsat*Sc1 - mu_0*Ha*Sa,
        k_disp2*(mu_rec*mu_0*Hm2+Br)*Sm2 - Bsat*Sc2,
        k_disp3*(mu_rec*mu_0*Hm3+Br)*Sm3 - Bsat*Sc3,
        
    )

In [152]:
alpha_0 = pi/2 - alpha_ar- 3*alpha_mag - alpha_dente
Ha = Bsat/mu_0

In [158]:
def equations(X):
    Hm1,Hm2,Hm3,Rs1,Rs2,Rs3,Rc1,Rc2,Rc3,H_p = X
    #Áreas:
    
    #Sr1 = (Ro-Ri)/3*L
    #Sr2 = Sr1
    #Sr3 = Sr1
    
    #Sg1 = alpha_mag*(Rg+Ro)/2*L
    #Sg2 = Sg1
    #Sg3 = Sg1
    
    #Sm1 = alpha_mag*(Rs1+Rg)/2*L
    #Sm2 = alpha_mag*(Rs2+Rg)/2*L
    #Sm3 = alpha_mag*(Rs3+Rg)/2*L
    
    #Sc1 = (Rc1-Rs1)/4*L
    #Sc2 = (Rc2-Rs2)/4*L
    #Sc3 = (Rc3-Rs3)/4*L
    
    #Sa = (k_a*Rso-Rg)*L
    #Sb = Sa
    
    #alpha_0 = pi/2 - alpha_ar- 3*alpha_mag - alpha_dente
    
    #Comprimentos:
    
    #Lr1 = (2*(alpha_0 + alpha_dente+alpha_ar+1*alpha_mag/2)*(Ri+3*(Ro-Ri)/4) + 2*1*(Ro-Ri)/4)
    #Lr2 = (2*(alpha_0 + alpha_dente+alpha_ar+3*alpha_mag/2)*(Ri+2*(Ro-Ri)/4) + 2*2*(Ro-Ri)/4)
    #Lr3 = (2*(alpha_0 + alpha_dente+alpha_ar+5*alpha_mag/2)*(Ri+1*(Ro-Ri)/4) + 2*3*(Ro-Ri)/4)
    
    #Lg1 = (Rg - Ro)
    #Lg2 = Lg1
    #Lg3 = Lg1
    
    #Lm1 = (Rs1 - Rg)
    #Lm2 = (Rs2 - Rg)
    #Lm3 = (Rs3 - Rg)
        
    #Lc1 = (2*(alpha_0 + alpha_dente+alpha_ar+1*alpha_mag/2)*(Rs1+1*(Rc1-Rs1)/4) + 2*1*(Rc1-Rs1)/4)
    #Lc2 = (2*(alpha_0 + alpha_dente+alpha_ar+3*alpha_mag/2)*(Rs2+2*(Rc2-Rs2)/4) + 2*2*(Rc2-Rs2)/4)
    #Lc3 = (2*(alpha_0 + alpha_dente+alpha_ar+5*alpha_mag/2)*(Rs3+3*(Rc3-Rs3)/4) + 2*3*(Rc3-Rs3)/4)
    
    #Lp = 2*(alpha_0+alpha_dente)*(Rso+Rg)/2
    #La = alpha_ar*(Rso+Rg)/2 
    #Lb=La    
    
    return(
        2*Hm1*(Rs1 - Rg) + 2*(Rg - Ro)*Hg1+ Hr1*(2*(alpha_0 + alpha_dente+alpha_ar+1*alpha_mag/2)*(Ri+3*(Ro-Ri)/4) + 2*1*(Ro-Ri)/4)+ (2*(alpha_0 + alpha_dente+alpha_ar+1*alpha_mag/2)*(Rs1+3*(Rc1-Rs1)/8) + 2*3*(Rc1-Rs1)/8)*Hc1,
        2*Hm2*(Rs2 - Rg) + 2*(Rg - Ro)*Hg2+ Hr2*(2*(alpha_0 + alpha_dente+alpha_ar+3*alpha_mag/2)*(Ri+2*(Ro-Ri)/4) + 2*2*(Ro-Ri)/4)+ (2*(alpha_0 + alpha_dente+alpha_ar+3*alpha_mag/2)*(Rs2+5*(Rc2-Rs2)/8) + 2*5*(Rc2-Rs2)/8)*Hc2,
        2*Hm3*(Rs3 - Rg) + 2*(Rg - Ro)*Hg3+ Hr3*(2*(alpha_0 + alpha_dente+alpha_ar+5*alpha_mag/2)*(Ri+1*(Ro-Ri)/4) + 2*3*(Ro-Ri)/4)+ (2*(alpha_0 + alpha_dente+alpha_ar+5*alpha_mag/2)*(Rs3+7*(Rc3-Rs3)/8) + 2*7*(Rc3-Rs3)/8)*Hc3,
        
        2*(Rs1 - Rg)*Hm1 + 2*Ha*(alpha_ar*(Rso+Rg)/2) + 2*(alpha_0+alpha_dente)*(Rso+Rg)/2*Hp + Hc1*(2*(alpha_0 + alpha_dente+alpha_ar+1*alpha_mag/2)*(Rs1+1*(Rc1-Rs1)/8) + 2*1*(Rc1-Rs1)/8),
        #2*(Rs1 - Rg)*Hm1 + 2*Ha*alpha_ar*(Rg) + 2*(alpha_0+alpha_dente)*(Rg)*Hp + Hc1*(2*(alpha_0 + alpha_dente+alpha_ar+1*alpha_mag/2)*(Rs1+1*(Rc1-Rs1)/4) + 2*1*(Rc1-Rs1)/4),
        
        (mu_rec*mu_0*Hm1+Br)*alpha_mag*(Rs1+Rg)/2 -Bgap[0]*alpha_mag*(Rg+Ro)/2 - Bsat*(Rso-Rg),
        #(mu_rec*mu_0*Hm1+Br)*alpha_mag*(Rs1+Rg)/2 - Bgap[0]*alpha_mag*(Rg+Ro)/2,
        (mu_rec*mu_0*Hm2+Br)*alpha_mag*(Rs2+Rg)/2 - Bgap[1]*alpha_mag*(Rg+Ro)/2,
        (mu_rec*mu_0*Hm3+Br)*alpha_mag*(Rs3+Rg)/2 - Bgap[2]*alpha_mag*(Rg+Ro)/2,
        
        #B_r1*Sr1 - Bgap[0]*Sg1,
        #B_r2*Sr2 - Bgap[1]*Sg2,
        #B_r3*Sr3 - Bgap[2]*Sg3,
        
        #mu_0*Ha*(k_a*Rso-Rg)*L - B_sat*(Rso-Rg)*L,
        #mu_0*Ha- Bsat,
        #Rso -320,
        
        (mu_rec*mu_0*Hm1+Br)*alpha_mag*(Rs1+Rg)/2 -Bsat*(Rc1-Rs1)/4 - *(Rso-Rg),
        #(mu_rec*mu_0*Hm1+Br)*alpha_mag*(Rs1+Rg)/2 - Bsat*(Rc1-Rs1)/4,
        (mu_rec*mu_0*Hm2+Br)*alpha_mag*(Rs2+Rg)/2 - Bsat*(Rc2-Rs2)/4,
        (mu_rec*mu_0*Hm3+Br)*alpha_mag*(Rs3+Rg)/2 - Bsat*(Rc3-Rs3)/4,
        
    )

In [159]:
from scipy.optimize import minimize

In [160]:
Hm1,Hm2,Hm3,Rs1,Rs2,Rs3,Rc1,Rc2,Rc3,Rso = minimize(equations,np.array(-350164,-300000,-300000,Rg+150,Rg+150,Rg+150,Rg+250,Rg+250,Rg+250,Rg+50))

ValueError: only 2 non-keyword arguments accepted

In [161]:
x0 = np.array([-350164,-300000,-300000,Rg+150,Rg+150,Rg+150,Rg+250,Rg+250,Rg+250,Rg+50])


In [162]:
(mu_rec*mu_0*Hm1+Br)*alpha_mag*(Rs1+Rg)/2,Bsat*(Rc1-Rs1)/4,Bsat*(Rso-Rg)

(45.16039439736458, 45.160394395353265, 234.0)

In [163]:
Hm1,Hm2,Hm3,Rs1,Rs2,Rs3,Rc1,Rc2,Rc3,Rso = fsolve(equations,x0,maxfev = 1000000000)

In [164]:
mu_rec*mu_0*Hm1+Br,Br

(0.22235686660225706, 1.41)

-48.93076919290859

In [128]:
Hm1,Hm2,Hm3,Rs1,Rs2,Rs3,Rc1,Rc2,Rc3,Rso

(-1651289.8429597947,
 -329225.3337770487,
 -332763.3420410535,
 215.29174874268267,
 296.69945204435027,
 298.8965432714897,
 2093.4523809976686,
 630.2091018246454,
 531.4613167644713,
 91.64506266661887)

In [ ]:
Sr1 = (Ro-Ri)/3*L
Sr2 = Sr1
Sr3 = Sr1

Sg1 = alpha_mag*(Rg+Ro)/2*L
Sg2 = Sg1
Sg3 = Sg1

Sm1 = alpha_mag*(Rs1+Rg)/2*L
Sm2 = alpha_mag*(Rs2+Rg)/2*L
Sm3 = alpha_mag*(Rs3+Rg)/2*L

Sc1 = (Rc1-Rs1)/4*L
Sc2 = (Rc2-Rs2)/4*L
Sc3 = (Rc3-Rs3)/4*L

Sa = (k_a*Rso-Rg)*L
Sb = Sa

alpha_0 = pi/2 - alpha_ar- 3*alpha_mag - alpha_dente

#Comprimentos:
Lr1 = 2*(alpha_0 + alpha_dente+alpha_ar+1*alpha_mag/2)*(Ri+3*(Ro-Ri)/4) + 2*1*(Ro-Ri)/4
Lr2 = 2*(alpha_0 + alpha_dente+alpha_ar+3*alpha_mag/2)*(Ri+2*(Ro-Ri)/4) + 2*2*(Ro-Ri)/4
Lr3 = 2*(alpha_0 + alpha_dente+alpha_ar+5*alpha_mag/2)*(Ri+1*(Ro-Ri)/4) + 2*3*(Ro-Ri)/4

Lg1 = Rg - Ro
Lg2 = Lg1
Lg3 = Lg1

Lm1 = Rs1 - Rg
Lm2 = Rs2 - Rg
Lm3 = Rs3 - Rg

Lc1 = 2*(alpha_0 + alpha_dente+alpha_ar+1*alpha_mag/2)*(Rs1+1*(Rc1-Rs1)/4) + 2*1*(Rc1-Rs1)/4
Lc2 = 2*(alpha_0 + alpha_dente+alpha_ar+3*alpha_mag/2)*(Rs2+2*(Rc2-Rs2)/4) + 2*2*(Rc2-Rs2)/4
Lc3 = 2*(alpha_0 + alpha_dente+alpha_ar+5*alpha_mag/2)*(Rs3+3*(Rc3-Rs3)/4) + 2*3*(Rc3-Rs3)/4

Lp = 2*(alpha_0+alpha_dente)*(Rso+Rg)/2
La = alpha_ar*(Rso+Rg)/2 
Lb=La 

In [ ]:
2*Hm1*Lm1+2*Lg1*Hg1+Hr1*Lr1+Lc1*Hc1,
2*Hm2*Lm2+2*Lg2*Hg2+Hr1*Lr2+Lc2*Hc2,
2*Hm3*Lm3+2*Lg3*Hg3+Hr1*Lr3+Lc3*Hc3,

2*Lm1*Hm1 + 2*Ha*La + Lp*Hp + Hc1*Lc1,

k_disp1*(mu_rec*mu_0*Hm1+Br)*Sm1 - Bgap[0]*Sg1 - mu_0*Ha*Sa,
k_disp2*(mu_rec*mu_0*Hm2+Br)*Sm2 - Bgap[1]*Sg2,
k_disp3*(mu_rec*mu_0*Hm3+Br)*Sm3 - Bgap[2]*Sg3,

#B_r1*Sr1 - Bgap[0]*Sg1,
#B_r2*Sr2 - Bgap[1]*Sg2,
#B_r3*Sr3 - Bgap[2]*Sg3,

mu_0*Ha*(k_a*Rso-Rg)*L - B_sat*(Rso-Rg)*L,


k_disp1*(mu_rec*mu_0*Hm1+Br)*Sm1 - Bsat*Sc1 - mu_0*Ha*Sa,
k_disp2*(mu_rec*mu_0*Hm2+Br)*Sm2 - Bsat*Sc2,
k_disp3*(mu_rec*mu_0*Hm3+Br)*Sm3 - Bsat*Sc3,

In [ ]:
2*(alpha_0 + alpha_dente+alpha_ar+1*alpha_mag/2)*(Rs1+1*(Rc1-Rs1/4)) + 2*1*(Rc1-Rs1/4)

In [ ]:
Rs1

In [ ]:
Lr1,Lr2,Lr3